In [ ]:
#Dependent Libraries
!pip install pandas==1.5.3
!pip install scikit-learn==1.2.1
!pip install keras==2.12.0

In [ ]:
#Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
# Read data from Excel CSV file
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/transformed_data.csv')
# Create DataFrame
df = pd.DataFrame(data)

# Prepare features and target
ftr = df.iloc[:,:-1] # Excludes file path columns and target feature
target = df['DeepFake']  # Target Feature

In [ ]:
# Splitting dataset to training and testing
ftr_train, ftr_test, target_train, target_test = train_test_split(ftr, target, test_size=0.2, random_state=42)

In [ ]:
# Model definition with updated layer configuration and dropout rates
model = Sequential()

# Input layer with increased width
model.add(Dense(256, input_dim=ftr_train.shape[1]))  # Increased width
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

# Additional hidden layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

# Third hidden layer
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Adjusting the learning rate in the optimizer
#optimizer = Adam(learning_rate=0.001)
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

In [ ]:
# Compile the model with the custom optimizer
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# Define the path where you want to save the model
model_path = '/content/drive/My Drive/Colab Notebooks/MLP_ML/Models/MLP_best_model_extended_ftr.h5'

model_checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

In [ ]:
# Train the model
model.fit(ftr_train, target_train, epochs=150, batch_size=32, validation_split=0.2,
          callbacks=[early_stopping, model_checkpoint, reduce_lr])

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(ftr_test, target_test)
print(f'Test accuracy: {accuracy:.2%}')

355/355 [==============================] - 1s 1ms/step - loss: 0.3623 - accuracy: 0.8426
Test accuracy: 84.26%


In [ ]:
#--optional--Loading model and test accuracy using split datasets ftr_test and target_test
from tensorflow.keras.models import load_model
# Load the saved model
best_model = load_model(model_path)

# Evaluate the model
loss, accuracy = best_model.evaluate(ftr_test, target_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")